In [ ]:
############################################################
#	All code taken from https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
#	Some modifications have been made for the purppose of this project
############################################################

import numpy
# import spacy
# import gensim
import sys
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
# nltk.download('punkt')
# from gensim.models import KeyedVectors
# from gensim.test.utils import common_texts
# from gensim.models import Word2Vec
# import en_core_web_lg

In [ ]:
file = open("mqd2.txt").read()
file = file.lower()

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk import FreqDist
tok = RegexpTokenizer('\w+|[\w+.,]')
data = tok.tokenize(file)

#data = data[0:int(len(data)/2)]
data =" ".join(data)


In [ ]:
# freq = FreqDist(data)
# for w in range(len(data)):
#   if freq[data[w]] < 3:
#     data[w] = 'other'

In [ ]:
words = sorted(list(set(data)))
words

[' ',
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
n_words = len(data)
print(n_words)
n_vocab = len(words)
print(n_vocab)

1430746
39


In [ ]:
# nlp = en_core_web_lg.load()

In [ ]:
# from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

# wordList =[]
# vectorList = []
# for key in nlp.vocab.strings:
#   wordList.append(key)
#   vectorList.append(nlp.vocab[key].vector)

# kv = WordEmbeddingsKeyedVectors(nlp.vocab.vectors_length)

# kv.add(wordList, vectorList)
# #kv.most_similar

In [ ]:
word_to_num = dict((c, i) for i, c in enumerate(words))
#word_to_num

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
	seq_in = data[i:i + seq_length]
	seq_out = data[i + seq_length]
	dataX.append([word_to_num[w] for w in seq_in])
	dataY.append(word_to_num[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1430546


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
#print(X[0])
#print(y[0])

In [ ]:
...
# define the LSTM model
model = Sequential()
model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
...
# define the checkpoint
filepath="CHARS-iqg-checkpoint-s200-n1024.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
 103/5589 [..............................] - ETA: 20:46 - loss: 3.1805

KeyboardInterrupt: ignored

In [ ]:
...
# load the network weights
filename = "iqg-model-seq3-200epochs-1024n.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
num_to_word = dict((i, c) for i, c in enumerate(words))

In [ ]:

...
# pick a random seed
# seed = "if i had a dollar for every time".split()
# seed = [word_to_num[w] for w in seed]
# pattern = seed
a1 = []
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([num_to_word[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #print('\n', get_top_n_words(prediction, 3))
    index = numpy.argmax(prediction)
    #index = random_top_n_word_index(prediction, 1)
    result = num_to_word[index]
    seq_in = [num_to_word[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" t is sub "
hect . the best whr oaresten and the best whr oaresten and the best whr oaresten and the best whr oa
Done.


In [ ]:
def get_top_n_words(arr, n):
  arr_r = arr.reshape(-1)
  top_n = arr_r.argsort()[-n:][::-1]
  word_arr = [num_to_word[w] for w in top_n]
  return word_arr

def random_top_n_word_index(arr, n):
  r = numpy.random.randint(0, n)
  arr_r = arr.reshape(-1)
  top_n = arr_r.argsort()[-n:][::-1]
  return top_n[r]

#print(a1)
#print(get_top_n_words(a1 , 2))